# Loop 81 Analysis: Breaking the CV-LB Line

## Critical Situation
- 81 experiments completed, 12 LB submissions
- Best CV: 0.0083 (exp_030) → Best LB: 0.0877
- Target: 0.0347 (152.8% gap)
- **ALL approaches fall on the same CV-LB line: LB = 4.29*CV + 0.0528**
- **Intercept (0.0528) > Target (0.0347) → IMPOSSIBLE to reach target with current approaches**

## Key Insight
The intercept represents STRUCTURAL DISTRIBUTION SHIFT. The test solvents are fundamentally different from training solvents in ways that tabular features don't capture.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# All submissions with CV and LB scores
submissions = [
    ('exp_000', 0.0111, 0.0982),
    ('exp_001', 0.0123, 0.1065),
    ('exp_003', 0.0105, 0.0972),
    ('exp_005', 0.0104, 0.0969),
    ('exp_006', 0.0097, 0.0946),
    ('exp_007', 0.0093, 0.0932),
    ('exp_009', 0.0092, 0.0936),
    ('exp_012', 0.0090, 0.0913),
    ('exp_024', 0.0087, 0.0893),
    ('exp_026', 0.0085, 0.0887),
    ('exp_030', 0.0083, 0.0877),
    ('exp_035', 0.0098, 0.0970),
]

cv_scores = np.array([s[1] for s in submissions])
lb_scores = np.array([s[2] for s in submissions])

# Fit linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(cv_scores, lb_scores)

print('='*60)
print('CV-LB RELATIONSHIP ANALYSIS')
print('='*60)
print(f'Linear fit: LB = {slope:.4f} * CV + {intercept:.4f}')
print(f'R² = {r_value**2:.4f} (VERY STRONG FIT)')
print(f'\nIntercept: {intercept:.4f}')
print(f'Target LB: 0.0347')
print(f'\nRequired CV to hit target: (0.0347 - {intercept:.4f}) / {slope:.4f} = {(0.0347 - intercept) / slope:.4f}')
print('\n*** IMPOSSIBLE: Required CV is NEGATIVE ***')

In [ ]:
# Visualize the CV-LB relationship
plt.figure(figsize=(10, 6))

# Plot submissions
plt.scatter(cv_scores, lb_scores, c='blue', s=100, label='Submissions', zorder=5)

# Plot fitted line
cv_range = np.linspace(0, 0.015, 100)
lb_pred = slope * cv_range + intercept
plt.plot(cv_range, lb_pred, 'r--', label=f'Fit: LB = {slope:.2f}*CV + {intercept:.4f}')

# Plot target
plt.axhline(y=0.0347, color='green', linestyle=':', linewidth=2, label='Target LB = 0.0347')

# Plot intercept
plt.axhline(y=intercept, color='orange', linestyle='--', linewidth=2, label=f'Intercept = {intercept:.4f}')

plt.xlabel('CV Score (MSE)', fontsize=12)
plt.ylabel('LB Score (MSE)', fontsize=12)
plt.title('CV-LB Relationship: All Approaches Fall on Same Line', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 0.015)
plt.ylim(0.03, 0.12)
plt.tight_layout()
plt.savefig('/home/code/exploration/cv_lb_relationship.png', dpi=150)
plt.show()

print('\nKEY INSIGHT: The intercept (0.0528) is HIGHER than the target (0.0347).')
print('This means even with PERFECT CV=0, the expected LB would be 0.0528.')
print('The gap is STRUCTURAL - it represents distribution shift that cannot be fixed by model tuning.')

In [ ]:
# What approaches have been tried?
print('='*60)
print('APPROACHES TRIED (81 experiments)')
print('='*60)

approaches = [
    ('MLP variants', '50+', 'All fall on same CV-LB line'),
    ('LightGBM', '10+', 'Same line'),
    ('XGBoost', '10+', 'Same line'),
    ('CatBoost', '5+', 'Same line'),
    ('Random Forest', '3+', 'Same line'),
    ('Gaussian Process', '3+', 'Same line'),
    ('Ridge Regression', '2+', 'Same line'),
    ('GNN (attempted)', '2', 'CV=0.0256, 0.0196 - WORSE than tabular'),
    ('ChemBERTa (attempted)', '2', 'CV=0.0147 - WORSE than tabular'),
    ('ens-model kernel replica', '1', 'CV=0.0092 - Same line'),
]

for approach, count, result in approaches:
    print(f'{approach:30s} | {count:5s} | {result}')

print('\n' + '='*60)
print('CRITICAL: All tabular approaches fall on the SAME CV-LB line!')
print('GNN and ChemBERTa attempts performed WORSE than tabular baselines.')
print('='*60)

In [ ]:
# What might break the CV-LB line?
print('='*60)
print('STRATEGIES TO BREAK THE CV-LB LINE')
print('='*60)

strategies = [
    ('1. Proper GNN implementation', 
     'Previous GNN attempts had model class mismatch issues. Need to verify submission cells match CV.'),
    ('2. Extrapolation detection + conservative predictions',
     'Detect when test sample is far from training distribution, blend toward mean.'),
    ('3. Pseudo-labeling',
     'Use confident test predictions to augment training for distribution adaptation.'),
    ('4. Adversarial validation features',
     'Features that distinguish train/test can guide prediction calibration.'),
    ('5. Domain constraints',
     'Physics-based constraints that hold even on unseen solvents.'),
    ('6. Submit pending experiments',
     'exp_049, exp_050, exp_053 have CV=0.0081 - might have different CV-LB relationship.'),
]

for strategy, description in strategies:
    print(f'\n{strategy}')
    print(f'  → {description}')

In [ ]:
# Analyze pending submissions
print('='*60)
print('PENDING SUBMISSIONS (not yet submitted to LB)')
print('='*60)

pending = [
    ('exp_049', 0.0081, 'CatBoost+XGBoost ensemble'),
    ('exp_050', 0.0081, 'CatBoost+XGBoost fixed'),
    ('exp_052', 0.0109, 'CatBoost+XGBoost clipped'),
    ('exp_053', 0.0081, 'Exact template'),
    ('exp_054', 0.0085, 'Mixall approach'),
    ('exp_055', 0.0085, 'Minimal submission'),
    ('exp_057', 0.0093, 'Ens-model all features'),
    ('exp_063', 0.0112, 'Correct final cell'),
    ('exp_079', 0.0110, 'Mixall GroupKFold'),
]

print('\nExperiment | CV Score | Expected LB (based on line) | Description')
print('-'*80)
for exp, cv, desc in pending:
    expected_lb = slope * cv + intercept
    print(f'{exp:10s} | {cv:.4f}   | {expected_lb:.4f}                    | {desc}')

print('\nBest pending: exp_049, exp_050, exp_053 with CV=0.0081')
print(f'Expected LB: {slope * 0.0081 + intercept:.4f}')
print(f'This is still {(slope * 0.0081 + intercept - 0.0347) / 0.0347 * 100:.1f}% above target.')

In [ ]:
# Key insight: The benchmark paper achieved MSE 0.0039
print('='*60)
print('BENCHMARK PAPER ANALYSIS')
print('='*60)

print('\nThe benchmark paper (arXiv:2512.19530) achieved MSE 0.0039 using:')
print('  1. Graph Attention Networks (GAT) for molecular representation')
print('  2. Differential Reaction Fingerprints (DRFP)')
print('  3. Learned mixture-aware encodings')
print('  4. Hybrid architecture combining GNN + DRFP')

print('\nOur GNN attempts achieved CV=0.0256 and CV=0.0196 - 3-6x WORSE than tabular!')
print('This suggests our GNN implementations have fundamental issues.')

print('\nPossible issues with our GNN implementations:')
print('  1. Model class mismatch between CV and submission cells')
print('  2. Not using proper molecular graph structure')
print('  3. Not using attention mechanisms (GAT)')
print('  4. Not combining with DRFP features')
print('  5. Insufficient training (epochs, learning rate)')

In [ ]:
# Final recommendation
print('='*60)
print('RECOMMENDED NEXT STEPS')
print('='*60)

print('\n1. SUBMIT exp_049 or exp_053 (CV=0.0081) to verify CV-LB relationship')
print('   - If LB ≈ 0.0875 (as predicted), confirms we\'re on the same line')
print('   - If LB < 0.085, there might be hope with this approach')

print('\n2. IMPLEMENT EXTRAPOLATION DETECTION')
print('   - Add features measuring distance to training distribution')
print('   - When extrapolation detected, blend predictions toward mean')
print('   - This could REDUCE THE INTERCEPT')

print('\n3. IMPLEMENT PROPER GNN')
print('   - Use PyTorch Geometric with GATConv')
print('   - Verify submission cells use the SAME model class as CV')
print('   - Combine with DRFP features')

print('\n4. DO NOT continue optimizing tabular models')
print('   - All tabular approaches fall on the same CV-LB line')
print('   - Improving CV just moves along the line, not toward target')

In [ ]:
# Record finding
print('\n' + '='*60)
print('FINDING TO RECORD')
print('='*60)
print('Loop 81 Analysis: CV-LB relationship is LB = 4.29*CV + 0.0528 (R²=0.95).')
print('CRITICAL: Intercept (0.0528) > Target (0.0347). Required CV to hit target is -0.0042 (IMPOSSIBLE).')
print('Best LB: 0.0877 (exp_030). All 81 experiments fall on same CV-LB line.')
print('GNN attempts (CV=0.0256, 0.0196) performed WORSE than tabular (CV=0.0083).')
print('Must implement extrapolation detection or proper GNN to break the CV-LB line.')